<img src = "logo_neuralearn.png" height="100" width= "100" >
 <h1> <center> <font color = "black"> [Deep Learning with TensorFlow 2]   <br><font color="grey"> PARKINSON DISEASE DETECTION </font></center></h1>

In [1]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2, ResNet50
from tensorflow.keras.layers import Conv2D, Conv1D, BatchNormalization, LayerNormalization, Conv2DTranspose, Dropout, MaxPooling2D, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

<H1> DATA PREPARATION </H1>

In [2]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, data, batch_size, IN_1, IN_2, shuffle = False):
        
        self.data = data
        self.batch_size = batch_size
        self.data_list = os.listdir(data)
        self.IN_1 = IN_1
        self.IN_2 = IN_2
        
    def __len__(self):
        return int(np.floor(len(self.data_list)/self.batch_size) )

    def __getitem__(self, idx):
        
        X, y = self.__data_generation(idx)
        
        return X,y
    
    def get_spec(self, inp):
    
        zero_padding = (12000 - len(inp))*[0]
        zero_padding = tf.constant(zero_padding, tf.float32)
        inp = tf.cast(inp, tf.float32)
        equal_length = tf.concat([inp, zero_padding], 0)

        spectrogram = tf.signal.stft(
          inp, frame_length = 512, frame_step = 258)

        spectrogram = tf.abs(spectrogram)

        return np.resize(spectrogram, (self.IN_1,self.IN_2))


    def __data_generation(self, idx):
        
        X_1, X_2, X_4 = [],[],[]
        X = []
        y = []
        
        for j in range(idx*self.batch_size, (idx+1)*self.batch_size ):

            data_list = []
            f = open(self.data + self.data_list[idx], "r")
            for x in f:
                data_list.append([int(x) for x in x[:-1].split(';')])
            data_list = np.array(data_list, dtype = float)
            
            
            for i in range(7):
                data_list[:,i] = (data_list[:,i] - np.mean(data_list[:,i]))/np.std(data_list[:,i])
                
            X_1.append(self.get_spec(tf.constant(data_list)[:,0]))
            X_2.append(self.get_spec(tf.constant(data_list)[:,1]))
            #X_3.append(self.get_spec(tf.constant(data_list)[:,2]))
            X_4.append(self.get_spec(tf.constant(data_list)[:,3]))
            #X_5.append(self.get_spec(tf.constant(data_list)[:,4]))
            #X_6.append(self.get_spec(tf.constant(data_list)[:,5]))
            #X_7.append(self.get_spec(tf.constant(data_list)[:,6]))
            
            if(self.data_list[idx][0] == 'C'):
                y.append(0)
            else:
                y.append(1)
            
            
        return [tf.convert_to_tensor(X_1), tf.convert_to_tensor(X_2),tf.convert_to_tensor(X_4)], tf.convert_to_tensor(y)

In [3]:
train_images = 'D:/Machine_Learning/datasets/PARKINSON_HW/parkinson/pos/'
val_images = 'D:/Machine_Learning/datasets/PARKINSON_HW/parkinson/val_pos/'

IN_1, IN_2 = 32, 257
BATCH_SIZE = 2
EPOCH = 20

In [4]:
train_gen = DataGenerator(train_images, BATCH_SIZE, IN_1, IN_2)
val_gen = DataGenerator(val_images, BATCH_SIZE, IN_1, IN_2)

<h1>MODELING</h1>

In [5]:
class ConvLayers(tf.keras.Model):

    def __init__(self, NUM_FILTERS):
        super(ConvLayers, self).__init__()
        self.conv = Conv1D(NUM_FILTERS, 5, padding = 'same', activation ='relu',)
        
    def call(self, x, ):
        
        x = self.conv(x)
        
        return x

In [6]:
inputs = [tf.keras.Input(shape=(IN_1, IN_2, 1)) for _ in range(3)]

#inp = tf.keras.Input(shape = (IN_1, IN_2))


x_out = []

for i in range(1):
    
    #x = tf.keras.layers.experimental.preprocessing.Normalization()(inputs[i])
    
    x = ConvLayers(4)(inputs[i])
    x = ConvLayers(8)(x)
    x = Flatten()(x)

    x_out.append(x)


x_out = tf.concat(x, axis = -1)

x_out = Dense(4,)(x_out)
x_out = Dense(1, activation = 'sigmoid')(x_out)

model = tf.keras.Model(inputs = inputs, outputs = x_out)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 257, 1)] 0                                            
__________________________________________________________________________________________________
conv_layers (ConvLayers)        (None, 32, 257, 4)   24          input_1[0][0]                    
__________________________________________________________________________________________________
conv_layers_1 (ConvLayers)      (None, 32, 257, 8)   168         conv_layers[0][0]                
__________________________________________________________________________________________________
flatten (Flatten)               (None, 65792)        0           conv_layers_1[0][0]              
_______________________________________________________________________________________

<H1> TRAINING AND OPTIMIZATION </H1>

In [7]:
LR = 1e-3
model.compile(
    loss = tf.keras.losses.BinaryCrossentropy(),
    optimizer = Adam(lr = LR,),
    metrics = 'accuracy',
    #run_eagerly = True,
)

In [8]:
checkpoint_filepath = 'parkinson/cnn_dense.hdf5'

callback = tf.keras.callbacks.ModelCheckpoint(
    
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# Train model on dataset
history = model.fit(train_gen, validation_data = val_gen, shuffle = True, epochs = EPOCH, callbacks = [callback])

Epoch 1/20
31/31 [==============================] - 4s 144ms/step - loss: 3.5165 - accuracy: 0.4839 - val_loss: 6.6315 - val_accuracy: 0.4286
Epoch 2/20
31/31 [==============================] - 4s 124ms/step - loss: 1.0632 - accuracy: 0.8387 - val_loss: 1.2606 - val_accuracy: 0.7143
Epoch 3/20
31/31 [==============================] - 4s 123ms/step - loss: 0.9817 - accuracy: 0.8065 - val_loss: 1.8667 - val_accuracy: 0.5714
Epoch 4/20
31/31 [==============================] - 4s 125ms/step - loss: 0.2204 - accuracy: 0.9032 - val_loss: 3.8722 - val_accuracy: 0.5714
Epoch 5/20
31/31 [==============================] - 4s 123ms/step - loss: 0.0215 - accuracy: 1.0000 - val_loss: 1.1862 - val_accuracy: 0.8571
Epoch 6/20
31/31 [==============================] - 4s 132ms/step - loss: 0.0493 - accuracy: 0.9677 - val_loss: 2.9373 - val_accuracy: 0.7143
Epoch 7/20
31/31 [==============================] - 4s 125ms/step - loss: 0.0771 - accuracy: 0.9677 - val_loss: 0.7654 - val_accuracy: 0.8571
Epoch 

KeyboardInterrupt: 

<H1> TESTING </H1>

In [9]:
def detect(test_image):

    X_1 = []

    data_list = []

    def get_spec(inp):
    
        zero_padding = (12000 - len(inp))*[0]
        zero_padding = tf.constant(zero_padding, tf.float32)
        inp = tf.cast(inp, tf.float32)
        equal_length = tf.concat([inp, zero_padding], 0)

        spectrogram = tf.signal.stft(
          inp, frame_length = 512, frame_step = 258)

        spectrogram = tf.abs(spectrogram)

        return np.resize(spectrogram, (IN_1,IN_2))


    f = open(test_image, "r")
    for x in f:
        data_list.append([int(x) for x in x[:-1].split(';')])
    data_list = np.array(data_list, dtype = float)

    data_list[:,0] = (data_list[:,0] - np.mean(data_list[:,0]))/np.std(data_list[:,0])

    X_1.append(get_spec(tf.constant(data_list)[:,0]))


    return model.predict(tf.constant(X_1))

In [11]:
test_images = 'D:/Machine_Learning/datasets/PARKINSON_HW/parkinson/pos/'

for im in os.listdir(test_images):
    print(im)
    print(detect(test_images + im))

C_0001.txt
[[1.4855279e-07]]
C_0002.txt
[[0.00186903]]
C_0003.txt
[[9.878938e-05]]
C_0004.txt
[[7.065187e-05]]
C_0005.txt
[[9.06771e-06]]
C_0006.txt
[[0.00026376]]
C_0007.txt
[[0.00237635]]
C_0008.txt
[[5.1599986e-06]]
C_0009.txt
[[3.334745e-07]]
C_0010.txt
[[1.7307245e-06]]
C_0011.txt
[[8.8618335e-06]]
H_P000-0003.txt
[[0.99590814]]
H_P000-0015.txt
[[0.9988977]]
H_P000-0016.txt
[[0.9999964]]
H_p000-0017.txt
[[1.]]
H_p000-0018.txt
[[0.99999976]]
H_P000-0019.txt
[[0.9999579]]
H_P000-0020.txt
[[0.99999976]]
H_P000-0021.txt
[[0.9999999]]
H_P000-0022.txt
[[0.9999982]]
H_P000-0023.txt
[[1.]]
H_P000-0024.txt
[[0.99999666]]
H_P000-0025.txt
[[0.9951067]]
H_p000-0028.txt
[[1.]]
H_P000-0029.txt
[[0.99997914]]
H_P000-0030.txt
[[0.9999995]]
H_P000-0031.txt
[[1.]]
H_P000-0032.txt
[[0.9999987]]
H_P000-0033.txt
[[1.]]
H_P000-0034.txt
[[1.]]
H_P000-0035.txt
[[1.]]
H_P000-0036.txt
[[0.9937406]]
H_P000-0037.txt
[[1.]]
H_P000-0039.txt
[[1.]]
H_P000-0040.txt
[[1.]]
H_p000-0041.txt
[[1.]]
H_p000-0042.txt
[